In [7]:
from darts import TimeSeries
import datetime
import pandas as pd
from darts.models import KalmanForecaster
from darts.models.forecasting.xgboost import XGBModel

from darts.dataprocessing.transformers import MissingValuesFiller
data_filler = MissingValuesFiller()

In [3]:
df = pd.read_csv('../data/processed/R80711_imputed.csv')
cols = ['Ba_avg', 'P_avg', 'Q_avg', 'Ya_avg', 'Yt_avg', 'Ws1_avg', 'Ws2_avg',
        'Ws_avg', 'Wa_avg', 'Va_avg', 'Ot_avg', 'Rs_avg', 'Rbt_avg', 'Rm_avg','temp', 
        'pressure', 'humidity', 'wind_speed', 'wind_deg', 'rain_1h','snow_1h']
df['Date_time'] = pd.to_datetime(df['Date_time'], utc=True).dt.tz_localize(None)

In [8]:
series = TimeSeries.from_dataframe(df, time_col='Date_time', value_cols=cols, fill_missing_dates=True, freq=None)
train, val = series.split_after(pd.Timestamp('2017-01-01 00:00:00'))
train = data_filler.transform(train)
val = data_filler.transform(val)

In [11]:
model = KalmanForecaster(dim_x=10)
model.fit(train[:20000])

In [290]:
import darts.utils.timeseries_generation as tg
from darts.models import KalmanForecaster
import matplotlib.pyplot as plt

series1 = tg.sine_timeseries(value_frequency=0.05, length=100) + 0.1 * tg.gaussian_timeseries(length=100)
series2 = tg.sine_timeseries(value_frequency=0.02, length=100) + 0.2 * tg.gaussian_timeseries(length=100)

multivariate_series = series1.stack(series2)

model = KalmanForecaster(dim_x=4)
model.fit(multivariate_series)
pred = model.predict(n=50, num_samples=100)

In [293]:
train[0:10]

<TimeSeries (DataArray) (Date_time: 10, component: 21, sample: 1)>
array([[[-1.0000000e+00],
        [ 1.0726500e+03],
        [ 1.4490000e+01],
        [ 1.6144000e+02],
        [ 1.2730000e+01],
        [ 8.7700005e+00],
        [ 9.1599998e+00],
        [ 8.9600000e+00],
        [ 1.6525000e+02],
        [ 3.7800000e+00],
        [ 5.0900002e+00],
        [ 1.7180000e+01],
        [ 2.8709999e+01],
        [ 5.6902798e+03],
        [ 5.3900000e+00],
        [ 1.0110000e+03],
        [ 7.5000000e+01],
        [ 5.6600000e+00],
        [ 1.8000000e+02],
        [ 0.0000000e+00],
...
        [ 1.0811000e+03],
        [ 1.0270000e+01],
        [ 1.7789999e+02],
        [ 1.9809999e+01],
        [ 8.4200001e+00],
        [ 8.4899998e+00],
        [ 8.4600000e+00],
        [ 1.7012000e+02],
        [-7.7600002e+00],
        [ 5.7900000e+00],
        [ 1.7170000e+01],
        [ 2.8500000e+01],
        [ 5.7368101e+03],
        [ 5.7000000e+00],
        [ 1.0110000e+03],
        [ 6.5000000e+01],
        [ 5.6600000e+00],
        [ 1.9000000e+02],
        [ 0.0000000e+00],
        [ 0.0000000e+00]]])
Coordinates:
  * Date_time  (Date_time) datetime64[ns] 2012-12-31T23:00:00 ... 2013-01-01T...
  * component  (component) object 'Ba_avg' 'P_avg' ... 'rain_1h' 'snow_1h'
Dimensions without coordinates: sample
Attributes:
    static_covariates:  None
    hierarchy:          None